In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Mean Squared Error

## MSE Calculation

In [4]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt

loss_object = nn.MSELoss()

batch_size = 32
predictions = torch.randn(batch_size, 1)
labels = torch.randn(batch_size, 1)

mse = loss_object(predictions, labels)
print("MSE(torch): ", mse.item())

mse_manual = torch.mean(torch.pow(labels - predictions, 2))

print("MSE(manual): ", mse_manual.item())


MSE(torch):  1.4898978471755981
MSE(manual):  1.4898978471755981


## MSE with Model/Dataset

In [8]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

N, n_feature = 100, 5
batch_size = 32

X = torch.randn(N, n_feature)
Y = torch.randn(N, 1)


# 하나의 Dense Layer와 MSE를 계산하는 모델 설계
class Dense(torch.nn.Module):
    def __init__(self):
        super(Dense, self).__init__()
        self.layer = nn.Linear(5, 1)
        self.mse = nn.MSELoss()
        
    def forward(self, x, y):
        x = self.layer(x)
        x = self.mse(x, y)
        return x

# Dataset 클래스를 상속받아 커스텀 데이터셋 클래스 생성
class custom_dataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
    
# 인스터스(데이터) 생성
train_data = custom_dataset(X, Y)

# 만들어진 데이터가 배치형태로 만들어줘야하니까 DataLoader에다가 넣어줌
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)

model = Dense()


for x, y in train_loader:
    #print(x.shape, y.shape)
    loss = model(x, y)
    print(loss.detach().numpy())

#mse_manual = torch.mean(torch.pow(labels - predictions, 2))

#print("MSE(manual): ", mse_manual.item())




1.4717449
1.7970073
1.3212402
0.58484834


# Binary Cross Entropy

## BCE Calculation

In [59]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

batch_size = 32
n_class = 2

predictions = torch.rand((batch_size, 1))

# randint's 2nd arg -> One above the highest integer to be drawn from the distribution.
labels = torch.randint(0, n_class , (batch_size, 1))

loss_object = nn.BCELoss()
labels = labels.type(torch.FloatTensor)
loss = loss_object(predictions, labels)

# BCE: -[ylog(y^) + (1 - y)log(1 - y^)]
bce_manual = -(labels * torch.log(predictions) + (1 - labels) * torch.log(1 - predictions))
bce_manual = torch.mean(bce_manual)

print("BCE(Torch): ", loss.detach().numpy())
print("BCE(Manual): ", bce_manual.detach().numpy())




BCE(Torch):  1.0998785
BCE(Manual):  1.0998785


## BCE with Model/Dataset

In [69]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np
import matplotlib.pyplot as plt

N, n_features = 100, 5 

# Binary Classification을 위한 예제 Dataset 생성
torch_weights = torch.tensor([1, 2, 3, 4, 5], dtype = torch.float32)
torch_bias = torch.tensor([10], dtype = torch.float32)

X = torch.randn(N, n_features)
#print(X)
Y = torch.sum(torch_weights * X, 1) + torch_bias
#print(Y)
Y = (Y > 5).type(torch.int32)
Y = Y.type(torch.FloatTensor)
# 추후에 진행될 BCE loss 연산을 위해 2차원 데이터로 만들어준다.
Y = Y.unsqueeze(1) 


# 하나의 Dense Layer와 MSE를 계산하는 모델 설계
class Dense(torch.nn.Module):
    def __init__(self):
        super(Dense, self).__init__()
        self.layer = nn.Linear(5, 1)
        self.sigmoid = nn.Sigmoid()
        
        
    def forward(self, x):
        x = self.layer(x)
        x = self.sigmoid(x)
        return x
    
model = Dense()
loss_object = nn.BCELoss()


# Dataset 클래스를 상속받아 커스텀 데이터셋 클래스 생성
class custom_dataset(Dataset):
    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len
    
    
# 인스터스(데이터) 생성
train_data = custom_dataset(X, Y)

# 만들어진 데이터가 배치형태로 만들어줘야하니까 DataLoader에다가 넣어줌
train_loader = DataLoader(train_data, batch_size = batch_size, shuffle = True)


for x, y in train_loader:
    predictions = model(x)
    loss = loss_object(predictions, y)
    print(loss.item())
    


0.7864617705345154
0.7628585696220398
0.8226035833358765
0.8106143474578857
